In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import layers

import datetime
import tensorflow_addons as tfa 
import pandas as pd
import matplotlib.pylab as plt
import numpy as np

/opt/anaconda3/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.4.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


## Load data

In [2]:
import read_rijksdata
MIN_NUM_ARTWORK = 300

In [3]:
img_folder = '/Users/erebor/Downloads/out_img'

In [4]:
images, labels_onehot, labels, names = read_rijksdata.load_data(MIN_NUM_ARTWORK=MIN_NUM_ARTWORK,
                                                 img_folder =img_folder,
                                                 labels_file='labels.txt',
                                                 names_file='names.txt')

/Users/erebor/repos/ML_Rijksmuseum/read_rijksdata.py:22: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  names = pd.read_csv(names_file,delimiter = '/t',header=None)


 |███████████████████████████████████████-| 112038/112039 

Dataset loaded!
images shape: (29703, 56, 56, 3)
labels shape: (29703,)
labels (one-hot): (29703, 50)
names shape: (29703, 1)


In [5]:
print('# number of unique artists:', len(set(labels)))

# number of unique artists: 50


In [6]:
classes = len(list(set(labels)))
print('# of classes:',classes)

counts = pd.DataFrame(labels).value_counts()
print('Min # of artworks for all artists:',min(counts))
print('Min # of artworks specified:',MIN_NUM_ARTWORK)

# of classes: 50
Min # of artworks for all artists: 303
Min # of artworks specified: 300


## Download and compile model

In [7]:
input_shape = (56,56,3)
enet_kwargs = {'include_top':False,
               'weights':'imagenet',
               'input_tensor':None,
               'input_shape':input_shape,
               'pooling':None,
               'classes':classes,
               'classifier_activation':'softmax'}
enet_base = tf.keras.applications.efficientnet.EfficientNetB7(**enet_kwargs)
enet_base.trainable = True

enet_base.summary()

In [8]:
# set pre-trained model as base
enet = tf.keras.models.Sequential()
enet.add(enet_base)

enet.add(tf.keras.layers.GlobalMaxPooling2D())
enet.add(tf.keras.layers.Dropout(rate=0.01))
enet.add(tf.keras.layers.Dense(classes, activation="softmax"))

In [9]:
enet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb7 (Functional)  (None, 2, 2, 2560)        64097687  
_________________________________________________________________
global_max_pooling2d (Global (None, 2560)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2560)              0         
_________________________________________________________________
dense (Dense)                (None, 50)                128050    
Total params: 64,225,737
Trainable params: 63,915,010
Non-trainable params: 310,727
_________________________________________________________________


In [10]:
# loss function
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False,
                                               label_smoothing=0.0,
                                               name='categorical_crossentropy')

# metrics

TopKs = []
for k in [1,5,10,20]:
    TopK = tf.keras.metrics.TopKCategoricalAccuracy(k=k, name='top_{}'.format(k))
    TopKs.append(TopK)
metrics = ["acc"]
metrics.extend(TopKs)

f1 = tfa.metrics.F1Score(num_classes=classes, threshold=0.5)
metrics.append(f1)

# Optimizer
# very average Adam settings
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

#optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)
# compile it all
enet.compile(
    loss=loss,
    optimizer=optimizer,
    metrics=metrics)

## Train model

In [11]:
history = enet.fit(x=images,y=labels_onehot,validation_split=.20, epochs=5,batch_size=1000)

Epoch 1/5
24/24 [==============================] - 2785s 116s/step - loss: 0.0000e+00 - acc: 0.0486 - top_1: 0.0486 - top_5: 0.2740 - top_10: 0.5246 - top_20: 0.7590 - f1_score: 0.0000e+00 - val_loss: 0.0000e+00 - val_acc: 0.0510 - val_top_1: 0.0510 - val_top_5: 0.1655 - val_top_10: 0.6753 - val_top_20: 1.0000 - val_f1_score: 0.0000e+00
Epoch 2/5
24/24 [==============================] - 2658s 111s/step - loss: 0.0000e+00 - acc: 0.0110 - top_1: 0.0110 - top_5: 0.3919 - top_10: 0.7466 - top_20: 0.8775 - f1_score: 0.0000e+00 - val_loss: 0.0000e+00 - val_acc: 0.1862 - val_top_1: 0.1862 - val_top_5: 0.9968 - val_top_10: 0.9998 - val_top_20: 1.0000 - val_f1_score: 0.0000e+00
Epoch 3/5
24/24 [==============================] - 3194s 135s/step - loss: 0.0000e+00 - acc: 0.0113 - top_1: 0.0113 - top_5: 0.3836 - top_10: 0.8582 - top_20: 0.9627 - f1_score: 0.0000e+00 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00 - val_top_1: 0.0000e+00 - val_top_5: 0.0045 - val_top_10: 0.9128 - val_top_20: 1.0000 - 

In [12]:
# Save the weights
checkpoint_time = '{date:%Y-%m-%d_%H-%M}'.format(date=datetime.datetime.now())

save_file = './checkpoints/enet_{}'.format(checkpoint_time)
print('Saving to:',save_file)
enet.save_weights(save_file)

Saving to: ./checkpoints/enet_2022-05-03_01-47


In [ ]:
enet.save('models/enet.h5')

In [ ]:
feed them into a feedforward network after efficient net

## Re-load model and evaluation

In [ ]:
reconstructed_model = keras.models.load_model('models/enet.h5')

In [ ]:
allgood = np.testing.assert_allclose(enet.predict(X_), reconstructed_model.predict(X_))

In [ ]:
print(allgood)

In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

confusion_matrix = sklearn.metrics.confusion_matrix(y_true=Y, y_pred=y_pred, labels=None)

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(classes):
    fpr[i], tpr[i], _ = roc_curve(Y_train, y_pred)
    roc_auc[i] = auc(fpr[i], tpr[i])